In [1]:
import pandas as pd
import numpy as np
import time
import os
import progressbar

In [2]:
import sys
sys.path.insert(0, '/media/andrii/earth/Katia/CS_MasterThesis/Red_links_Project_for_Wiki_draft/py')
import functions

### Load en_wiki

In [3]:
#path to data
PATH_TO_DATA = '/media/andrii/earth/Katia/CS_MasterThesis/data/'
PATH_TO_DATA_UK = PATH_TO_DATA+"ukwiki/"
PATH_TO_DATA_EN = PATH_TO_DATA+"enwiki/"

In [4]:
data_time = '20180920'

In [5]:
ENWIKI_ART_FNMS = functions.list_from_dir(PATH_TO_DATA_EN+'enwiki_20180920/art_red/', 'art.csv.gz', [])
len(ENWIKI_ART_FNMS)

55

In [6]:
df_en, df_red = functions.load_csvs_to_df(PATH_TO_DATA_EN+'enwiki_20180920/art_red/', ENWIKI_ART_FNMS)

55
1/55
/media/andrii/earth/Katia/CS_MasterThesis/data/enwiki/enwiki_20180920/art_red/enwiki-20180920-pages-meta-current14.xml-p6197598p7697598.bz2_art.csv.gz
Total time: 0.3 minutes


In [7]:
print(df_en.shape)
print(df_en.columns)

(4553133, 2)
Index(['id', 'link_id'], dtype='object')


In [8]:
# load en article names

In [9]:
EN_ID_NAME = "enwiki-"+data_time+"-id_name.csv"

In [10]:
df_en_name = pd.read_csv(PATH_TO_DATA_EN+'enwiki_20180920/'+EN_ID_NAME, encoding='UTF-8', quotechar="\"", sep='^')
df_en_name = df_en_name[['id','title']]
print(df_en_name.shape)
print(df_en_name.columns)

(5719743, 2)
Index(['id', 'title'], dtype='object')


### Load uk_wiki

In [11]:
UKWIKI_ART_FNMS = functions.list_from_dir(PATH_TO_DATA_UK+'ukwiki_20180920/', 'pages-links.csv.gz', [])

In [12]:
UKWIKI_ART_FNMS

['ukwiki-20180920-pages-links.csv.gz']

In [13]:
df_uk_blue, df_uk_red = functions.load_csvs_to_df(PATH_TO_DATA_UK+'ukwiki_20180920/', UKWIKI_ART_FNMS)

1
1/1
/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/ukwiki_20180920/ukwiki-20180920-pages-links.csv.gz
Total time: 1.3 minutes


In [14]:
print(df_uk_blue.shape)
print(df_uk_blue.columns)

print(df_uk_red.shape)
print(df_uk_red.columns)

(19818824, 2)
Index(['id', 'link_id'], dtype='object')
(2874954, 2)
Index(['id', 'link_val'], dtype='object')


In [15]:
# load uk article names

In [16]:
UK_ID_NAME = "ukwiki-"+data_time+"-id_name.csv"
UK_ID_NAME

'ukwiki-20180920-id_name.csv'

In [17]:
df_uk_name = pd.read_csv(PATH_TO_DATA_UK+'ukwiki_20180920/'+UK_ID_NAME, sep='^')

print(df_uk_name.shape)
print(df_uk_name.columns)

(819212, 3)
Index(['id', 'title', 'length'], dtype='object')


In [18]:
# add ids to red links

red_links_ids = functions.ids_red_links(df_uk_name['id'], df_uk_red['link_val'])
red_links_ids['is_red_link'] = True

In [19]:
# create dataframe with titles and red link titles / add it to article titles df
df_uk_name['is_red_link'] = False
df_uk_name = pd.concat((df_uk_name,red_links_ids))
df_uk_name = df_uk_name.reset_index(drop=True)

print(df_uk_name.shape)
print(df_uk_name.columns)


(2382910, 4)
Index(['id', 'is_red_link', 'length', 'title'], dtype='object')


In [20]:
# Variant 1

# # add link ids to df_uk_red
# df_uk_red = df_uk_red.merge(right = red_links_ids, left_on = 'link_val', right_on = 'title', how = 'left')
# df_uk_red = df_uk_red[['id_x','id_y']]
# df_uk_red.columns = ['id', 'link_id']

In [23]:
# Variant 2

df_uk_red = pd.read_csv(PATH_TO_DATA_UK+'uk_red_links/'+'all_uk_redlinks_with_parent_ids.csv')

df_uk_red.head()

df_uk_red = df_uk_red[['id_parent', 'red_link_id']]
df_uk_red.columns = ['id', 'link_id']

/home/andrii/apps/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
# concatenate red and blue article to one df
df_uk = pd.concat((df_uk_red, df_uk_blue))

In [64]:
print(df_uk.shape)
#print(df_uk_blue.shape[0] + df_uk_red.shape[0])

(20867399, 2)


In [26]:
del df_uk_red, df_uk_blue

### Load uk-en correpondences

In [45]:
df_uk_name = pd.read_csv(PATH_TO_DATA_UK+'ukwiki_20180920/uk_names.csv.gz')
df_en_name = pd.read_csv(PATH_TO_DATA_EN+'enwiki_20180920/en_names.csv.gz')

In [18]:
df_uk_name.head()

,id,title,is_red_link,id_en
0,13,Об'єкт (філософія),False,18963910.0
1,13,Карл Ріттер,False,18963910.0
2,13,Александер фон Гумбольдт,False,18963910.0
3,13,Герард Меркатор,False,18963910.0
4,13,Мухаммад Аль-Ідрісі,False,18963910.0


In [19]:
df_en_name.head()

,id,title,id_uk
0,12,Anarchism,12101.0
1,25,Autism,37656.0
2,39,Albedo,10899.0
3,290,A,235422.0
4,303,Alabama,6320.0


In [25]:
# load encoding from file
en_nontranslated_encoding = np.load(PATH_TO_DATA+'en_nontranslated_encoding_draft.npy')
en_nontranslated_ids = np.load(PATH_TO_DATA+'en_nontranslated_ids_draft.npy')

In [36]:
en_nontranslated_ids

array([     344,      615,      643, ..., 57258258, 57715716, 58144667])

In [31]:
en_nontranslated_encoding

array([{6771078, 7558407, 7364232, 7354126, 7338001, 6963476, 7368729, 6712860, 7338041, 6349630, 7377736, 7353548, 7323731, 6750805, 7324000, 7368672, 7331300, 7571304, 7232746, 7443960, 6747004, 7353597, 6963455},
       {7441155, 7302788, 6669315, 6936068, 7441414, 7441292, 6614034, 7441178, 6913179, 7441439, 7441314, 7441331, 6236472, 7441468, 7441362, 7441493, 7440984, 6930664, 7440873, 7441386, 7441266, 7584249, 7501052},
       {6268548, 6481925, 6870788, 6275482, 7434910, 7073059, 7164323, 7542058, 6361135, 6804166, 6870983, 7038409, 6766156, 6558033, 6914005, 7078613, 6593111, 6835293, 7074660, 7561447, 6360939, 6663147, 6525805, 7178348, 6655090, 6220797},
       ...,
       {6815488, 7249665, 6737538, 7112706, 7187973, 7336074, 7618448, 7137688, 6761880, 6646682, 7220640, 7307810, 6643235, 6693796, 6746918, 7469478, 7664297, 6489651, 6457526, 7351351, 6626489, 7332793, 7497403, 6958397, 7209280, 7073473, 6246466, 6532674, 7079361, 6831561, 6659786, 7536459, 7661642, 7585741,

### Encode red links incoming en links

In [49]:
max_id = np.max(df_uk_name['id'])

In [73]:
df_uk[df_uk['link_id']].dtypes

KeyError: "['2817993' '2817994' '2817994' ... 2773289 340314 340314] not in index"

In [72]:
# select only red links from uk_wiki
max_id = np.max(df_uk_name['id'])
df_uk = df_uk[pd.notnull(df_uk['link_id'])]
df_uk_red = df_uk[df_uk['link_id']>max_id]
uk_red_ids = df_uk_red['link_id'].unique()
print('Number of red links in uk wiki: %d' %(len(uk_red_ids)))

TypeError: '<' not supported between instances of 'numpy.ndarray' and 'str'

In [41]:
df_uk_red = df_uk_red.drop_duplicates()

In [42]:
len(df_uk_red)

2443148

In [43]:
df_uk_red.head()

,id,link_id
0,13,2817993
1,13,2817994
2,1928599,2817994
3,13,2817995
4,584,2817996


In [35]:
df_uk_name.head()

,id,title,is_red_link,id_en
0,13,Об'єкт (філософія),False,18963910.0
1,13,Карл Ріттер,False,18963910.0
2,13,Александер фон Гумбольдт,False,18963910.0
3,13,Герард Меркатор,False,18963910.0
4,13,Мухаммад Аль-Ідрісі,False,18963910.0


In [45]:
ukredlinks_with_enids_sample = pd.read_csv(PATH_TO_DATA_UK+'ukredlinks_with_enids_sample.csv', sep='^')

In [95]:
ukredlinks_with_enids_sample.head()

,red_link_name,id_parent,red_link_id,id,title
0,Каунт Бейсі,625,2818114,34651.0,1904
1,Каунт Бейсі,36823,2818114,605856.0,WordPress
2,Каунт Бейсі,202525,2818114,50350.0,Ella Fitzgerald
3,Каунт Бейсі,202608,2818114,22519.0,Oscar Peterson
4,Каунт Бейсі,202613,2818114,2193014.0,Lester Young


In [48]:
# for every red link from the sample find its correponding en incoming link (only distinct)
df_uk_red_encoding = ukredlinks_with_enids_sample[['id', 'red_link_id']]
df_uk_red_encoding.columns = ['id_en','link_id']
df_uk_red_encoding = df_uk_red_encoding[~df_uk_red_encoding['id_en'].isnull()]
df_uk_red_encoding = df_uk_red_encoding.drop_duplicates()

In [52]:
df_uk_red_encoding.shape

(164526, 2)

In [50]:
df_uk_red_encoding.head()

,id_en,link_id
0,34651.0,2818114
1,605856.0,2818114
2,50350.0,2818114
3,22519.0,2818114
4,2193014.0,2818114


In [53]:
# for every red link from the sample calculate number of incoming links
df_uk_red_incoming = df_uk_red_encoding['link_id'].value_counts().reset_index()
df_uk_red_incoming.columns = ['id','n_incoming']

In [55]:
# select only those red links that can be encoded with at least 20 en incoming links
uk_red_ids = np.array(df_uk_red_incoming[df_uk_red_incoming['n_incoming']>=20]['id'])
print('Number of red links with at least 20 distinct incoming en links: %d' % (len(uk_red_ids)))

Number of red links with at least 20 distinct incoming en links: 3196


In [56]:
# encode top red links using incoming en links
uk_red_ids = np.sort(uk_red_ids)
df_uk_red_encoding = df_uk_red_encoding[df_uk_red_encoding['link_id'].isin(uk_red_ids)]
df_uk_red_encoding = df_uk_red_encoding.sort_values(by = 'link_id').reset_index(drop = True)
indices = np.array(df_uk_red_encoding.drop_duplicates(keep='first', subset=['link_id']).index)

uk_red_encoding = []
pbar = progressbar.ProgressBar()
for i in pbar(range(0, len(indices))):
    if i == len(indices)-1:
        this_encoding = set(df_uk_red_encoding.iloc[indices[i]:]['id_en'])
    else:
        this_encoding = set(df_uk_red_encoding.iloc[indices[i]:indices[i+1]]['id_en'])
    uk_red_encoding.append(this_encoding)

100% |########################################################################|


In [58]:
del df_uk_red_incoming, df_uk_red_encoding

In [57]:
# save encoding to file
np.save(file='uk_red_ids.npy', arr=uk_red_ids)
np.save(file='uk_red_encoding.npy', arr=uk_red_encoding)

In [12]:
# load encoding from file
uk_red_ids = np.load('uk_red_ids.npy')
uk_red_encoding = np.load('uk_red_encoding.npy')

In [81]:
df_uk_name.head()

,id,title,is_red_link,id_en
0,13,Об'єкт (філософія),False,18963910.0
1,13,Карл Ріттер,False,18963910.0
2,13,Александер фон Гумбольдт,False,18963910.0
3,13,Герард Меркатор,False,18963910.0
4,13,Мухаммад Аль-Ідрісі,False,18963910.0


In [112]:
len(red_links_names.drop_duplicates())

3196

In [82]:
print(len(list(df_uk_name[df_uk_name['id'].isin(uk_red_ids)]['title'])))

0


In [79]:
print(len(uk_red_ids))
print(len(list(df_uk_name[df_uk_name['id'].isin(uk_red_ids)]['title'])))
print(len(incoming_uk_link_names))
print(len(n_incoming_uk))
print(len(incoming_en_link_names))
print(len(n_incoming_en))

3196
0
3196
3196
3196
3196


In [77]:
n_incoming_en[1]

46

In [109]:
red_links_names = ukredlinks_with_enids_sample[ukredlinks_with_enids_sample['red_link_id'].isin(uk_red_ids)]['red_link_name'].to_frame()
red_links_names = red_links_names.drop_duplicates()
list_of_red_link = red_links_names['red_link_name'].tolist()

In [ ]:
# create df with incoming uk ids and names for every red link
incoming_uk_link_names = []
n_incoming_uk = []
incoming_en_link_names = []
n_incoming_en = []

pbar = progressbar.ProgressBar()
for i in pbar(range(len(uk_red_ids))):
    red_id = uk_red_ids[i]
    incoming_ids = list(df_uk_red[df_uk_red['link_id'] == red_id]['id'])
    incoming_names = set(df_uk_name[df_uk_name['id'].isin(incoming_ids)]['title'])
    incoming_en_names = set(df_en_name[df_en_name['id'].isin(uk_red_encoding[i])]['title'])
              
    incoming_uk_link_names.append(incoming_names)
    n_incoming_uk.append(len(incoming_names))
    incoming_en_link_names.append(incoming_en_names) 
    n_incoming_en.append(len(incoming_en_names))

In [117]:
df_red_links = pd.DataFrame({'red_link_id': uk_red_ids, 
                          'red_link_name': list_of_red_link,
                          'incoming_uk_names': incoming_uk_link_names,
                          'n_incoming_uk': n_incoming_uk,
                          'incoming_en_names': incoming_en_link_names,
                          'n_incoming_en': n_incoming_en
                          })
              
df_red_links = df_red_links[['red_link_id','red_link_name','incoming_uk_names',
                             'n_incoming_uk', 'incoming_en_names', 'n_incoming_en']]

In [119]:
print(df_red_links.shape)
df_red_links.head(100)

(3196, 6)


,red_link_id,red_link_name,incoming_uk_names,n_incoming_uk,incoming_en_names,n_incoming_en
0,2818114,Каунт Бейсі,"{кларнет, Джонні Ріверс, Баррі Гарріс, Money (...",1598,"{Billie Holiday, Al McKibbon, Bob Cranshaw, Bu...",42
1,2818231,аль-Каїм,"{папа Римський, Феодосій Печерський, Гослар, а...",552,"{1043, 1071, 1048, 1040, 1030s, 1062, 1032, 10...",46
2,2818265,королева,"{тайська мова, Станіслав Август Понятовський, ...",2985,"{Australia, Jeanne d'Albret, 1601, Joan II of ...",50
3,2821142,Prentice Hall,"{Бандама, Вировитиця, Метрополітен Ченду, Міні...",3614,"{Transport in Guernsey, Transport in Colombia,...",231
4,2824527,Tokar.ua,"{кадмій, Криптоанархізм, VoIP, Nissan Sunny, П...",1885,"{Hyperloop, Hromadske.TV, Volvo, Hearing loss,...",37
5,2824727,Monthly Notices of the Royal Astronomical Society,"{Хромосфера, Жирафа (сузір'я), григоріанський ...",2016,"{List of galaxies, WASP-2b, WASP-1, Atmosphere...",57
6,2825668,Світ (видавництво),"{тайська мова, Горючі корисні копалини, Карлос...",10421,"{Transport in Guernsey, Demographics of Tonga,...",472
7,2825773,Дубович Іван Андрійович,"{тайська мова, Горючі корисні копалини, Карлос...",9621,"{Transport in Guernsey, Demographics of Tonga,...",448
8,2826419,помірні повітряні маси,"{безодня Моллой, Таганрозький морський торгове...",4132,"{Geography of Algeria, Sea of the Hebrides, Sc...",65
9,2826499,Доповідь про торгівлю людьми,"{тайська мова, Горючі корисні копалини, бемба,...",5955,"{Demographics of Madagascar, Demographics of T...",200


In [120]:
# save red links incoming links to files
df_red_links.to_csv(PATH_TO_DATA+'red_links_summary_jaccard_model.csv.gz', compression='gzip', header=True, index=False)

In [20]:
# load 
df_red_links = pd.read_csv(PATH_TO_DATA+'red_links_summary_jaccard_model.csv.gz')

### Find most similar uk articles for red links

In [124]:
en_article_list = []
en_article_found = []

pbar = progressbar.ProgressBar()
for j in pbar(range(len(uk_red_ids))):
    red_link = uk_red_ids[j]
    red_link_encoding = uk_red_encoding[j]
    similarities = []
    for i in range(len(en_nontranslated_ids)):
        similarity = functions.jaccard(red_link_encoding, en_nontranslated_encoding[i])
        similarities.append(similarity)
    
    # select top 5 most similar uk article, zero similarities excluded
    similarities = np.array(similarities)
    indices = np.argsort(-similarities)[:5]
    score = np.round(similarities[indices],3)
    n_nonzero = np.sum(score>0)
    indices = np.sort(indices[:n_nonzero])

    # scores and article names sorted by article ids
    score = np.round(similarities[indices],3)
    en_article_ids = en_nontranslated_ids[indices]
    en_article_names = list(df_en_name[df_en_name['id'].isin(en_article_ids)]['title'])
    
    # sort by jaccard similarity
    d = dict(zip(en_article_names, score))
    d_sorted = sorted(((value, key) for (key,value) in d.items()), reverse=True)
    
    # save to list
    en_article_list.append(d_sorted)
    
    if len(en_article_names)>0:
        en_article_found.append(True)
    else:
        en_article_found.append(False)

        
# create df to show results
df_similarities = pd.DataFrame({'red_link_id': uk_red_ids,
                   'en_similar_found': en_article_found, 
                   'en_similar': en_article_list
                  })
    
    

100% |########################################################################|


In [19]:
# version for getting all candidates


en_article_list = []
en_article_found = []

pbar = progressbar.ProgressBar()
for j in pbar(range(len(uk_red_ids))):
    red_link = uk_red_ids[j]
    red_link_encoding = uk_red_encoding[j]
    similarities = []
    for i in range(len(en_nontranslated_ids)):
        similarity = functions.jaccard(red_link_encoding, en_nontranslated_encoding[i])
        similarities.append(similarity)
    
    # zero similarities excluded
    similarities = np.array(similarities)
    indices = np.argsort(-similarities)
    score = np.round(similarities[indices],3)
    n_nonzero = np.sum(score>0)
    indices = np.sort(indices[:n_nonzero])

    # scores and article names sorted by article ids
    score = np.round(similarities[indices],3)
    en_article_ids = en_nontranslated_ids[indices]
    en_article_names = list(df_en_name[df_en_name['id'].isin(en_article_ids)]['title'])
    
    # sort by jaccard similarity
    d = dict(zip(en_article_names, score))
    d_sorted = sorted(((value, key) for (key,value) in d.items()), reverse=True)
    
    # save to list
    en_article_list.append(d_sorted)
    
    if len(en_article_names)>0:
        en_article_found.append(True)
    else:
        en_article_found.append(False)

        
# create df to show results
df_similarities = pd.DataFrame({'red_link_id': uk_red_ids,
                   'en_similar_found': en_article_found, 
                   'en_similar': en_article_list
                  })
    
    

100% |########################################################################|


In [21]:
df_red_results = df_red_links.merge(right=df_similarities, right_on = 'red_link_id',
                                     left_on = 'red_link_id', how = 'left')

In [22]:
df_red_results.head()

,red_link_id,red_link_name,incoming_uk_names,n_incoming_uk,incoming_en_names,n_incoming_en,en_similar,en_similar_found
0,2818114,Каунт Бейсі,"{'кларнет', 'Джонні Ріверс', 'Баррі Гарріс', ""...",1598,"{'Billie Holiday', 'Al McKibbon', 'Bob Cransha...",42,"[(0.081, The Count Basie Story), (0.076, The A...",True
1,2818231,аль-Каїм,"{'папа Римський', 'Феодосій Печерський', 'Госл...",552,"{'1043', '1071', '1048', '1040', '1030s', '106...",46,"[(0.031, Ahmad al-Muqtadir), (0.03, Cai Xiang)...",True
2,2818265,королева,"{'тайська мова', 'Станіслав Август Понятовськи...",2985,"{'Australia', ""Jeanne d'Albret"", '1601', 'Joan...",50,"[(0.033, List of consorts of the monarch of Ir...",True
3,2821142,Prentice Hall,"{'Бандама', 'Вировитиця', 'Метрополітен Ченду'...",3614,"{'Transport in Guernsey', 'Transport in Colomb...",231,"[(0.078, Chemical tanker), (0.067, Break of ga...",True
4,2824527,Tokar.ua,"{'кадмій', 'Криптоанархізм', 'VoIP', 'Nissan S...",1885,"{'Hyperloop', 'Hromadske.TV', 'Volvo', 'Hearin...",37,"[(0.05, List of virtual communities with more ...",True


In [128]:
df_red_results['en_similar_found'].value_counts()

True     3185
False      11
Name: en_similar_found, dtype: int64

In [24]:
# save results with all candidates to file
df_red_results.to_csv(PATH_TO_DATA+'uk_red_links_results_all_candidates.csv', header=True, index=False, encoding = 'UTF-8')

In [129]:
# save results to file
df_red_results.to_csv(PATH_TO_DATA+'uk_red_links_results.csv', header=True, index=False, encoding = 'UTF-8')

In [131]:
# load 
df_red_results = pd.read_csv(PATH_TO_DATA+'uk_red_links_results.csv', encoding = 'UTF-8')

In [23]:
df_red_results = df_red_results[['red_link_name', 'en_similar']]

In [134]:
# save results to file
df_red_results.to_csv(PATH_TO_DATA+'uk_red_links_results_lighter.csv', header=True, index=False, encoding = 'UTF-8')